If you are running this on Google Colab, you need to uncomment (remove the `#`) and execute the following lines to install the Pyomo package, the solver, and some helper tools. If you are running this on Binder or elsewhere (e.g. your own computer) you can ignore this.

In [1]:
# !pip install pyomo==6.4.1
# !apt install glpk-utils
# !pip install "git+https://github.com/sjpfenninger/sen1511.git#egg=sen1511utils&subdirectory=sen1511utils"

In [2]:
import pyomo.environ as pyo

from sen1511utils import summarise_results

<div class="alert alert-block alert-info">

💡 Tasks 1-3, 5-6, and 8 are solved on paper.

</div>

## Group tasks – robust optimisation

The consumer price for the robust optimization case is assumed to deviate at most by 40% of the nominal value (which was used for the definition of the deterministic case) as defined in the table below:

| Time Period| Price λ [$/kWh]|
|:---|---:|
|1|120|
|2|75|
|3|110|
|4|60|

with the exception of the first time period, when the price is perfectly known to the consumer.

### 4.

Calculate the optimal consumption schedule for deferrable load by solving the robust optimisation problem from question 3 in Python/Pyomo.

In [3]:
m = pyo.ConcreteModel(name = "Robust Model")
m.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

##
# 1. Decision variables
##

m.u1 = pyo.Var(domain=pyo.NonNegativeReals)
m.u2 = pyo.Var(domain=pyo.NonNegativeReals)
m.u3 = pyo.Var(domain=pyo.NonNegativeReals)
m.u4 = pyo.Var(domain=pyo.NonNegativeReals)
m.eps2 = pyo.Var(domain=pyo.NonNegativeReals)
m.eps3 = pyo.Var(domain=pyo.NonNegativeReals)
m.eps4 = pyo.Var(domain=pyo.NonNegativeReals)
m.beta = pyo.Var(domain=pyo.NonNegativeReals)

##
# 2. Objective function
##

m.obj = pyo.Objective(
    expr = (120*m.u1 - 100*m.u1 - 100*(m.u2 + m.u3 + m.u4) + m.eps2 + m.eps3 + m.eps4 + 2*m.beta + 75*m.u2 + 110*m.u3 + 60*m.u4),
    sense = pyo.minimize,
)


##
# 3. Constraints
##

m.epsbeta2 = pyo.Constraint(expr = m.eps2  + m.beta >= 30 * m.u2)
m.epsbeta3 = pyo.Constraint(expr = m.eps3  + m.beta >= 44 * m.u3)
m.epsbeta4 = pyo.Constraint(expr = m.eps4  + m.beta >= 24 * m.u4)

# Per-hour max constraint 
m.u1max = pyo.Constraint(expr=m.u1 <= 3)
m.u2max = pyo.Constraint(expr=m.u2 <= 3)
m.u3max = pyo.Constraint(expr=m.u3 <= 3)
m.u4max = pyo.Constraint(expr=m.u4 <= 3)

# Ramping
m.u0 = 0
m.ramp_up1   = pyo.Constraint(expr = m.u1 - m.u0 <= 1.5)  
m.ramp_down1 = pyo.Constraint(expr = m.u0 - m.u1 <= 1.5) 
m.ramp_up2   = pyo.Constraint(expr = m.u2 - m.u1 <= 1.5)  
m.ramp_down2 = pyo.Constraint(expr = m.u1 - m.u2 <= 1.5)
m.ramp_up3   = pyo.Constraint(expr = m.u3 - m.u2 <= 1.5)  
m.ramp_down3 = pyo.Constraint(expr = m.u2 - m.u3 <= 1.5)
m.ramp_up4   = pyo.Constraint(expr = m.u4 - m.u3 <= 1.5)  
m.ramp_down4 = pyo.Constraint(expr = m.u3 - m.u4 <= 1.5)

# Total Production
m.maxsum = pyo.Constraint(expr=m.u1+m.u2+m.u3+m.u4 <= 8)
m.minsum = pyo.Constraint(expr=m.u1+m.u2+m.u3+m.u4 >= 6)

# # Solve the problem
solver = pyo.SolverFactory('glpk')
solver.solve(m)


{'Problem': [{'Name': 'unknown', 'Lower bound': -16.7027027027027, 'Upper bound': -16.7027027027027, 'Number of objectives': 1, 'Number of constraints': 18, 'Number of variables': 9, 'Number of nonzeros': 36, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.009220123291015625}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [4]:
summarise_results(m)

,Name,Value
0,obj,-16.702703
,Name,Value
0,u1,0.283784
1,u2,1.783784
2,u3,1.216216
3,u4,2.716216
4,eps2,0.000000
5,eps3,0.000000
6,eps4,11.675676
7,beta,53.513514


### 7.

Calculate the optimal consumption schedule for the two different values of Γ: 0.8 and 1.2.

What can you conclude about the sensitivity of the optimal schedule to Γ?

In [5]:
m = pyo.ConcreteModel(name = "Robust Model with  Γ=1.2")
m.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

##
# 1. Decision variables
##

m.u1 = pyo.Var(domain=pyo.NonNegativeReals)
m.u2 = pyo.Var(domain=pyo.NonNegativeReals)
m.u3 = pyo.Var(domain=pyo.NonNegativeReals)
m.u4 = pyo.Var(domain=pyo.NonNegativeReals)
m.eps2 = pyo.Var(domain=pyo.NonNegativeReals)
m.eps3 = pyo.Var(domain=pyo.NonNegativeReals)
m.eps4 = pyo.Var(domain=pyo.NonNegativeReals)
m.beta = pyo.Var(domain=pyo.NonNegativeReals)

##
# 2. Objective function
##

m.obj = pyo.Objective(
    expr = (120*m.u1 - 100*m.u1 - 100*(m.u2 + m.u3 + m.u4) + m.eps2 + m.eps3 + m.eps4 + 1.2*m.beta + 75*m.u2 + 110*m.u3 + 60*m.u4),
    sense = pyo.minimize,
)


##
# 3. Constraints
##

m.epsbeta2 = pyo.Constraint(expr = m.eps2  + m.beta >= 30 * m.u2)
m.epsbeta3 = pyo.Constraint(expr = m.eps3  + m.beta >= 44 * m.u3)
m.epsbeta4 = pyo.Constraint(expr = m.eps4  + m.beta >= 24 * m.u4)

# Per-hour max constraint 
m.u1max = pyo.Constraint(expr=m.u1 <= 3)
m.u2max = pyo.Constraint(expr=m.u2 <= 3)
m.u3max = pyo.Constraint(expr=m.u3 <= 3)
m.u4max = pyo.Constraint(expr=m.u4 <= 3)

# Ramping
m.u0 = 0
m.ramp_up1   = pyo.Constraint(expr = m.u1 - m.u0 <= 1.5)  
m.ramp_down1 = pyo.Constraint(expr = m.u0 - m.u1 <= 1.5) 
m.ramp_up2   = pyo.Constraint(expr = m.u2 - m.u1 <= 1.5)  
m.ramp_down2 = pyo.Constraint(expr = m.u1 - m.u2 <= 1.5)
m.ramp_up3   = pyo.Constraint(expr = m.u3 - m.u2 <= 1.5)  
m.ramp_down3 = pyo.Constraint(expr = m.u2 - m.u3 <= 1.5)
m.ramp_up4   = pyo.Constraint(expr = m.u4 - m.u3 <= 1.5)  
m.ramp_down4 = pyo.Constraint(expr = m.u3 - m.u4 <= 1.5)

# Total Production
m.maxsum = pyo.Constraint(expr=m.u1+m.u2+m.u3+m.u4 <= 8)
m.minsum = pyo.Constraint(expr=m.u1+m.u2+m.u3+m.u4 >= 6)

# # Solve the problem
solver = pyo.SolverFactory('glpk')
solver.solve(m)

{'Problem': [{'Name': 'unknown', 'Lower bound': -60.8, 'Upper bound': -60.8, 'Number of objectives': 1, 'Number of constraints': 18, 'Number of variables': 9, 'Number of nonzeros': 36, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.012048006057739258}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [6]:
summarise_results(m)

,Name,Value
0,obj,-60.800000
,Name,Value
0,u1,0.700000
1,u2,2.200000
2,u3,1.500000
3,u4,3.000000
4,eps2,0.000000
5,eps3,0.000000
6,eps4,6.000000
7,beta,66.000000


In [7]:
m = pyo.ConcreteModel(name = "Robust Model with  Γ=0.8")
m.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

##
# 1. Decision variables
##

m.u1 = pyo.Var(domain=pyo.NonNegativeReals)
m.u2 = pyo.Var(domain=pyo.NonNegativeReals)
m.u3 = pyo.Var(domain=pyo.NonNegativeReals)
m.u4 = pyo.Var(domain=pyo.NonNegativeReals)
m.eps2 = pyo.Var(domain=pyo.NonNegativeReals)
m.eps3 = pyo.Var(domain=pyo.NonNegativeReals)
m.eps4 = pyo.Var(domain=pyo.NonNegativeReals)
m.beta = pyo.Var(domain=pyo.NonNegativeReals)

##
# 2. Objective function
##

m.obj = pyo.Objective(
    expr = (120*m.u1 - 100*m.u1 - 100*(m.u2 + m.u3 + m.u4) + m.eps2 + m.eps3 + m.eps4 + 0.8*m.beta + 75*m.u2 + 110*m.u3 + 60*m.u4),
    sense = pyo.minimize,
)


##
# 3. Constraints
##

m.epsbeta2 = pyo.Constraint(expr = m.eps2  + m.beta >= 30 * m.u2)
m.epsbeta3 = pyo.Constraint(expr = m.eps3  + m.beta >= 44 * m.u3)
m.epsbeta4 = pyo.Constraint(expr = m.eps4  + m.beta >= 24 * m.u4)

# Per-hour max constraint 
m.u1max = pyo.Constraint(expr=m.u1 <= 3)
m.u2max = pyo.Constraint(expr=m.u2 <= 3)
m.u3max = pyo.Constraint(expr=m.u3 <= 3)
m.u4max = pyo.Constraint(expr=m.u4 <= 3)

# Ramping
m.u0 = 0
m.ramp_up1   = pyo.Constraint(expr = m.u1 - m.u0 <= 1.5)  
m.ramp_down1 = pyo.Constraint(expr = m.u0 - m.u1 <= 1.5) 
m.ramp_up2   = pyo.Constraint(expr = m.u2 - m.u1 <= 1.5)  
m.ramp_down2 = pyo.Constraint(expr = m.u1 - m.u2 <= 1.5)
m.ramp_up3   = pyo.Constraint(expr = m.u3 - m.u2 <= 1.5)  
m.ramp_down3 = pyo.Constraint(expr = m.u2 - m.u3 <= 1.5)
m.ramp_up4   = pyo.Constraint(expr = m.u4 - m.u3 <= 1.5)  
m.ramp_down4 = pyo.Constraint(expr = m.u3 - m.u4 <= 1.5)

# Total Production
m.maxsum = pyo.Constraint(expr=m.u1+m.u2+m.u3+m.u4 <= 8)
m.minsum = pyo.Constraint(expr=m.u1+m.u2+m.u3+m.u4 >= 6)

# # Solve the problem
solver = pyo.SolverFactory('glpk')
solver.solve(m)

{'Problem': [{'Name': 'unknown', 'Lower bound': -89.4, 'Upper bound': -89.4, 'Number of objectives': 1, 'Number of constraints': 18, 'Number of variables': 9, 'Number of nonzeros': 36, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.012305259704589844}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [8]:
summarise_results(m)

,Name,Value
0,obj,-89.400000
,Name,Value
0,u1,0.900000
1,u2,2.400000
2,u3,1.500000
3,u4,3.000000
4,eps2,0.000000
5,eps3,0.000000
6,eps4,0.000000
7,beta,72.000000


## Group tasks – stochastic optimisation

In the “Required reading – Robust optimisation example” we looked at the problem of an electricity consumer facing both uncertain electricity price for next week (24*7= 168hrs) and addressed this decision problem with robust optimisation.
We now want to approach the same problem with stochastic programming. To do so we consider additionally that not only price, but also demand is uncertain. Both remain constant throughout the week. 
Scenario data for demand and price are provided in the table:

|Scenario data for the consumer||||
|:---|---:|---:|---:|
| Scenario # | Probability (per unit) | Demand [MW]| Price [€/MWh]|
|1|0.2|110|50|
|2|0.6|100|46|
|3|0.2|80 |44|

The rest of the problem remains the same: the consumer has the possibility of buying up to 90 MW at €45/MWh throughout next week, by signing a bilateral contract before next week, i.e., before knowing the actual demand and pool price it has to face.

The decision-making problem of this consumer can be formulated as a stochastic programming problem: the consumer has to decide how much to buy from the contract PC, and to decide his pool purchases for each of the three considered demand/price realizations (scenarios) P1, P2, and P3.

The objective function is the expected cost faced by the consumer to supply its uncertain demand.


### 9.

Solve the problem using Python/Pyomo and provide the objective function value and the optimal decisions.

In [9]:
model = pyo.ConcreteModel(name = "Stochastic Model")
model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

##
# 1. Decision variables
##

model.Pc = pyo.Var(domain=pyo.NonNegativeReals)
model.P1 = pyo.Var(domain=pyo.NonNegativeReals)
model.P2 = pyo.Var(domain=pyo.NonNegativeReals)
model.P3 = pyo.Var(domain=pyo.NonNegativeReals)

##
# 2. Objective function
##

model.profit = pyo.Objective(
    expr = (168 * (45 * model.Pc + 0.2 * 50 * model.P1 + 0.6 * 46 * model.P2 + 0.2 * 44 * model.P3)),
    sense = pyo.minimize,
)

##
# 3. Constraints
##

model.demand1 = pyo.Constraint(expr = model.Pc + model.P1 >=  110)
model.demand2= pyo.Constraint(expr = model.Pc + model.P2 >=  100)
model.demand3 = pyo.Constraint(expr = model.Pc + model.P3 >=  80)

model.LowerLimitc = pyo.Constraint(expr = model.Pc >= 0)
model.LowerLimit1 = pyo.Constraint(expr = model.P1 >= 0)
model.LowerLimit2 = pyo.Constraint(expr = model.P2 >= 0)
model.LowerLimit3 = pyo.Constraint(expr = model.P3 >= 0)

model.UpperLimit2 = pyo.Constraint(expr = model.Pc <= 90)

# Solve the problem
solver = pyo.SolverFactory('glpk')
solver.solve(model)


{'Problem': [{'Name': 'unknown', 'Lower bound': 747936.0, 'Upper bound': 747936.0, 'Number of objectives': 1, 'Number of constraints': 9, 'Number of variables': 5, 'Number of nonzeros': 12, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.009058952331542969}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [10]:
summarise_results(model)